# Analysis of dynamical polarizability of $CO$ in the linear response formalism

We analyze the computation of the dynamical polarizabilityi inside the linear response formalism.

The aim of this notebook is to:

* compute the Casida matrix associated to the highest value of virtual orbitals. 
* build the matrices associated to a lower number of virtual orbitals (by extracting its elements from the highest one) and discuss the rate of convergence of the statical polarizability to the reference value (formerly computed by including an explicit electric field). 
* evaluate the dynamical polarizability spectrum and discuss its stability in function of the number of virtual orbitals considered.  

In [1]:
%autosave 0
from BigDFT import Logfiles as lf
from BigDFT import Calculators as C
from futile import Yaml
import matplotlib.pyplot as plt
import numpy as np
import scipy.fftpack
import sys
sys.path.insert(0,'../')
import Routine as R
outputpath = 'linear-response/'

Autosave disabled


First of all we compute the coupling matrix

In [36]:
inp0 = {'dft' : 
        {'hgrids': 0.37,
         'gnrm_cv' : 1e-8,
         'itermax' : 100,
         'inputpsiid': 2},
        'tddft':{'tddft_approach': 'full'},
        'posinp': 'CO_posinp.xyz'}
Yaml.dump(inp0,outputpath + 'default.yaml')

In [41]:
virtMax = 256
rVal = 7.0

In [42]:
inp={'dft':
     {'rmult': [rVal, 9.0],
      'itermax_virt' : 1},
     'output': {
      'orbitals' : 'No',
      'verbosity' : 3}, 
     'radical': 'Rmult_7.0'}

inp['dft']['norbv'] = -virtMax
inpname = 'NumVirt_'+str(virtMax)
Yaml.dump(inp,outputpath + inpname + '.yaml')

Execute one input file per time and copy by hand the coupling matrix and transition dipoles in the "TDDFT-Results" folder with the name coupling_matrix_n and transition_dipoles_n

In [21]:
code=C.SystemCalculator(omp=2,mpi=4)
%cd $outputpath
code.run(name='NumVirt_16',skip=False,outdir='./log/')
%cd ../

/media/DATA/RICERCA/LINEAR RESPONSE/LR-nb/POLARIZABILITY/CO/linear-response
Executing command:  mpirun -np 4 $BIGDFT_ROOT/bigdft -n NumVirt_16 -d ./log/
/media/DATA/RICERCA/LINEAR RESPONSE/LR-nb/POLARIZABILITY/CO


Read the coupling matrix and dipoles for each value of the number of virtual orbitals

In [22]:
dipoles = []
C = []
%cd $outputpath
dpath = 'log/TDDFT-Results/transition_dipoles_*'
logs = !ls $dpath
print logs
for l in logs:
    dipoles.append(np.loadtxt(l))
listnorbs=[d.shape[0] for d in dipoles]
ipiv=np.argsort(listnorbs)
dipoles=[dipoles[i] for i in ipiv]

Cpath = 'log/TDDFT-Results/coupling_matrix_*'
logs = !ls $Cpath
print logs
for l in logs:
    C.append(np.loadtxt(l))
    
listnorbs=[c.shape[0] for c in C]
ipiv=np.argsort(listnorbs)
C=[C[i] for i in ipiv]
%cd ..

/media/DATA/RICERCA/LINEAR RESPONSE/LR-nb/POLARIZABILITY/CO/linear-response
['log/TDDFT-Results/transition_dipoles_16.txt']
['log/TDDFT-Results/coupling_matrix_16.txt']
/media/DATA/RICERCA/LINEAR RESPONSE/LR-nb/POLARIZABILITY/CO


In [25]:
dipoles[0][:,0] # the x component
print dipoles[0].shape[0]
print C[0].shape

160
(160, 160)


We observe that $C$ is a symmetric matrix

In [26]:
np.allclose(C[0],C[0].T,atol=1.e-12)

True